<h3>Pull Data from the NHL Data API</h3>

- code pulls all plays for all games in 2018 and 2019 seasons. 
- much refactoring to be done!

In [1]:
import numpy as np
import pandas as pd
import os
import io
import seaborn as sns
import requests
import json

import matplotlib.pyplot as plt
import pymc3 as pm

import scipy
import scipy.stats as stats
import scipy.optimize as opt
#import statsmodels.api as sm

import time
import datetime

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
plt.style.use('seaborn-darkgrid')
pd.set_option('display.max_columns', 500)
%matplotlib inline 

In [2]:
def get_game_results(year):
    #Get NHL Data for Last Season
    r = requests.get('https://statsapi.web.nhl.com/api/v1/schedule?stats=gameLog&season=' + str(year))
    data = r.json()
    gameType = []
    away = []
    home = []
    away_score = []
    home_score = []
    venue = []
    gameDate = []
    gameState = []
    gameid = []
    
    for date in data['dates']:
        for game in date['games']:
            gameType.append(game['gameType'])
            away.append(game['teams']['away']['team']['name'])
            home.append(game['teams']['home']['team']['name'])
            away_score.append(game['teams']['away']['score'])
            home_score.append(game['teams']['home']['score'])
            venue.append(game['venue']['name'])
            gameDate.append(str(date['date']))
            gameState.append(game['status']['statusCode'])
            gameid.append(game['gamePk'])
#         print("Done with " + str(date['date']))

    df = pd.DataFrame({'GameType': gameType, 'Away': away, 'Home': home, 'Final_Away_Score': away_score, 'Final_Home_Score': home_score,
                   'Venue': venue, 'GameDate': gameDate, 'GameState': gameState, 'GameID': gameid})
    
    return df

In [3]:
#Get Game Results for '17-'18 and '18-'19
game_results_18 = get_game_results('20172018')
game_results_19 = get_game_results('20182019')

In [4]:
#Limit Current Year to Only GameState == 6 or 7 (1 = scheduled)
game_results_19 = game_results_19[game_results_19['GameState'] != 1]

**Get the Play by Play data for every game in the data set.**
- To Do: refactor because this is long and gross!

In [5]:
def get_play_by_play(df):
    columns = df.columns.tolist() + ['EventID', 'Time_in_Period', 'Time_Remaining', 'Period',
       'Period_Type', 'x', 'y', 'Event_Description', 'Event',
       'Event_Type_ID', 'Home_Score', 'Away_Score', 'Player_1_Name',
       'Player_1_ID', 'Player_2_Name', 'Player_2_ID', 'Event_Team', 'Goalie', 'Goalie_ID',
        'Shooter', 'Shooter_ID', 'Scorer', 'Scorer_ID', 'Assist1', 'Assist1_ID', 'Assist2', 'Assist2_ID',
        'EmptyNet']
    
    master = pd.DataFrame(columns=columns)
    
    n = 0

    for game in df['GameID']:
        #Get Events for a game
        r = requests.get('https://statsapi.web.nhl.com/api/v1/game/'+ str(game) + '/feed/live')
        data = r.json()
        eventId = []
        periodTime = []
        periodTimeRemaining = []
        period = []
        periodType = []
        coord_x = []
        coord_y = []
        description = []
        event = []
        eventTypeID = []
        goals_home = []
        goals_away = []
        player_1 = []
        player_2 = []
        player_1_id = []
        player_2_id = []
        team = []
        goalie = []
        goalie_id = []
        shooter = []
        shooter_id = []
        scorer = []
        scorer_id = []
        assist_1 = []
        assist_1_id = []
        assist_2 = []
        assist_2_id = []
        empty_net = []
        shot_type = []

        for play in data['liveData']['plays']['allPlays']:
            try:
                eventId.append(play['about']['eventId'])
            except:
                eventId.append('None')
            try:
                periodTime.append(play['about']['periodTime'])
            except:
                periodTime.append(np.NaN)
            try:
                periodTimeRemaining.append(play['about']['periodTimeRemaining'])
            except:
                periodTimeRemaining.append(np.NaN)
            try:
                period.append(play['about']['period'])
            except:
                period.append('None')
            try:
                periodType.append(play['about']['periodType'])
            except:
                periodType.append('None')
            try:
                coord_x.append(play['coordinates']['x'])
            except:
                coord_x.append(np.NaN)
            try:
                coord_y.append(play['coordinates']['y'])
            except:
                coord_y.append(np.NaN)
            try:
                description.append(play['result']['description'])
            except:
                description.append('None')
            try:
                event.append(play['result']['event'])
            except:
                event.append('None')
            try:
                eventTypeID.append(play['result']['eventTypeId'])
            except:
                eventTypeID.append('None')
            try:
                goals_home.append(play['about']['goals']['home'])
            except:
                goals_home.append(np.NaN)
            try:
                goals_away.append(play['about']['goals']['away'])
            except:
                goals_away.append(np.NaN)
            try:
                player_1.append(play['players'][0]['player']['fullName'])
            except:
                player_1.append('None')
            try:
                player_1_id.append(play['players'][0]['player']['id'])
            except:
                player_1_id.append('None')
            try:
                player_2.append(play['players'][1]['player']['fullName'])
            except:
                player_2.append('None')
            try:
                player_2_id.append(play['players'][1]['player']['id'])
            except:
                player_2_id.append('None')

            #Append Empty Net Goal if Applicable (really just a safety collection)
            try:
                empty_net.append(play['result']['emptyNet'])
            except:
                empty_net.append('None')

            try:
                shot_type.append(play['result']['secondaryType'])
            except:
                shot_type.append('None')

            #Append Goalies if Applicable
            players = []
            players_id = []
            for i in range(0, 7):
                try:
                    if play['players'][i]['playerType'] == 'Goalie':
                        players.append(play['players'][i]['player']['fullName'])
                        players_id.append(play['players'][i]['player']['id'])
                except:
                    pass

            if len(players) == 1:
                goalie.append(players[0])
                goalie_id.append(players_id[0])
            elif len(players) == 0:
                goalie.append('None')
                goalie_id.append('None')
            else:
                goalie.append(str(players))
                goalie_id.append(str(players_id))

            #Append Scorers if Applicable
            players = []
            players_id = []
            for i in range(0, 7):
                try:
                    if play['players'][i]['playerType'] == 'Scorer':
                        players.append(play['players'][i]['player']['fullName'])
                        players_id.append(play['players'][i]['player']['id'])
                except:
                    pass

            if len(players) == 1:
                scorer.append(players[0])
                scorer_id.append(players_id[0])
            elif len(players) == 0:
                scorer.append('None')
                scorer_id.append('None')
            else:
                scorer.append(str(players))
                scorer_id.append(str(players_id))

            #Append Shooter if Applicable
            players = []
            players_id = []
            for i in range(0, 7):
                try:
                    if play['players'][i]['playerType'] == 'Shooter':
                        players.append(play['players'][i]['player']['fullName'])
                        players_id.append(play['players'][i]['player']['id'])
                except:
                    pass

            if len(players) == 1:
                shooter.append(players[0])
                shooter_id.append(players_id[0])
            elif len(players) == 0:
                shooter.append('None')
                shooter_id.append('None')
            else:
                shooter.append(str(players))
                shooter_id.append(str(players_id))

            #Append Assists if Applicable
            assists = []
            assists_id = []
            for i in range(0, 7):
                try:
                    if play['players'][i]['playerType'] == 'Assist':
                        assists.append(play['players'][i]['player']['fullName'])
                        assists_id.append(play['players'][i]['player']['id'])
                except:
                    pass

            if len(assists) > 0:
                assist_1.append(assists[0])
                assist_1_id.append(assists_id[0])

                if len(assists) > 1:
                    assist_2.append(assists[1])
                    assist_2_id.append(assists_id[1])
                else:
                    assist_2.append('None')
                    assist_2_id.append('None')
            else:
                assist_1.append('None')
                assist_1_id.append('None')
                assist_2.append('None')
                assist_2_id.append('None')


            try:
                team.append(play['team']['name'])
            except:
                team.append('None')


        plays_df = pd.DataFrame({'EventID': eventId, 'Time_in_Period': periodTime, 'Time_Remaining': periodTimeRemaining, 
                            'Period': period, 'Period_Type': periodType, 'x': coord_x, 'y': coord_y, 
                             'Event_Description': description, 'Event': event, 'Event_Type_ID': eventTypeID,
                            'Home_Score': goals_home, 'Away_Score': goals_away, 'Player_1_Name': player_1, 
                             'Player_1_ID': player_1_id, 'Player_2_Name': player_2, 'Player_2_ID': player_2_id, 'Event_Team': team, 
                                'Goalie': goalie, 'Goalie_ID': goalie_id, 'Shooter': shooter, 'Shooter_ID': shooter_id, 
                                'Scorer': scorer, 'Scorer_ID': scorer_id, 'Assist1': assist_1, 'Assist1_ID': assist_1_id,
                                'Assist2': assist_2, 'Assists2_ID': assist_2_id, 'ShotType': shot_type,'EmptyNet': empty_net})

        plays_df['GameID'] = game
        plays_df = plays_df.merge(df[df['GameID']==game], how='left', on='GameID')
        master = pd.concat([master, plays_df])
        print(str(game))
        print(n)
        n = n+1
    print('Done with Game Data')
    return master

In [6]:
#Get Play by Play Data
pbp_2018 = get_play_by_play(game_results_18)
pbp_2019 = get_play_by_play(game_results_19)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:232: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



2017010001
0
2017010003
1
2017010002
2
2017010005
3
2017010009
4
2017010010
5
2017010012
6
2017010011
7
2017010008
8
2017010004
9
2017010006
10
2017010017
11
2017010015
12
2017010016
13
2017010021
14
2017010014
15
2017010020
16
2017010018
17
2017010022
18
2017010019
19
2017010013
20
2017010025
21
2017010027
22
2017010028
23
2017010029
24
2017010030
25
2017010032
26
2017010024
27
2017010026
28
2017010031
29
2017010023
30
2017010038
31
2017010036
32
2017010037
33
2017010039
34
2017010034
35
2017010033
36
2017010035
37
2017010045
38
2017010046
39
2017010041
40
2017010043
41
2017010044
42
2017010040
43
2017010047
44
2017010042
45
2017010053
46
2017010050
47
2017010048
48
2017010049
49
2017010054
50
2017010055
51
2017010057
52
2017010052
53
2017010051
54
2017010056
55
2017010058
56
2017010062
57
2017010059
58
2017010061
59
2017010060
60
2017010063
61
2017010071
62
2017010067
63
2017010070
64
2017010072
65
2017010069
66
2017010073
67
2017010065
68
2017010068
69
2017010066
70
2017010064
71
20

2017020447
554
2017020448
555
2017020449
556
2017020450
557
2017020451
558
2017020452
559
2017020453
560
2017020454
561
2017020455
562
2017020456
563
2017020457
564
2017020458
565
2017020459
566
2017020460
567
2017020461
568
2017020462
569
2017020463
570
2017020464
571
2017020465
572
2017020466
573
2017020467
574
2017020468
575
2017020469
576
2017020470
577
2017020471
578
2017020472
579
2017020473
580
2017020474
581
2017020475
582
2017020476
583
2017020477
584
2017020478
585
2017020479
586
2017020480
587
2017020481
588
2017020482
589
2017020483
590
2017020484
591
2017020485
592
2017020486
593
2017020487
594
2017020488
595
2017020489
596
2017020490
597
2017020491
598
2017020492
599
2017020493
600
2017020494
601
2017020495
602
2017020496
603
2017020497
604
2017020498
605
2017020499
606
2017020500
607
2017020501
608
2017020503
609
2017020502
610
2017020504
611
2017020505
612
2017020506
613
2017020507
614
2017020508
615
2017020509
616
2017020510
617
2017020511
618
2017020512
619
2017020513

2017020985
1094
2017020986
1095
2017020987
1096
2017020988
1097
2017020990
1098
2017020989
1099
2017020991
1100
2017020992
1101
2017020993
1102
2017020994
1103
2017020995
1104
2017020996
1105
2017020997
1106
2017020998
1107
2017020999
1108
2017021000
1109
2017021001
1110
2017021002
1111
2017021003
1112
2017021004
1113
2017021005
1114
2017021006
1115
2017021007
1116
2017021008
1117
2017021009
1118
2017021010
1119
2017021011
1120
2017021012
1121
2017021013
1122
2017021014
1123
2017021016
1124
2017021015
1125
2017021017
1126
2017021018
1127
2017021019
1128
2017021020
1129
2017021021
1130
2017021022
1131
2017021023
1132
2017021024
1133
2017021025
1134
2017021026
1135
2017021027
1136
2017021028
1137
2017021029
1138
2017021030
1139
2017021031
1140
2017021032
1141
2017021033
1142
2017021034
1143
2017021035
1144
2017021036
1145
2017021037
1146
2017021038
1147
2017021039
1148
2017021040
1149
2017021041
1150
2017021042
1151
2017021043
1152
2017021044
1153
2017021045
1154
2017021046
1155
20170210

2018020047
156
2018020048
157
2018020049
158
2018020050
159
2018020051
160
2018020052
161
2018020053
162
2018020054
163
2018020055
164
2018020056
165
2018020057
166
2018020058
167
2018020059
168
2018020061
169
2018020060
170
2018020062
171
2018020063
172
2018020064
173
2018020065
174
2018020066
175
2018020067
176
2018020068
177
2018020069
178
2018020070
179
2018020071
180
2018020072
181
2018020073
182
2018020074
183
2018020075
184
2018020076
185
2018020077
186
2018020078
187
2018020079
188
2018020080
189
2018020081
190
2018020082
191
2018020083
192
2018020084
193
2018020085
194
2018020086
195
2018020087
196
2018020088
197
2018020089
198
2018020090
199
2018020091
200
2018020092
201
2018020093
202
2018020094
203
2018020095
204
2018020096
205
2018020097
206
2018020098
207
2018020099
208
2018020100
209
2018020101
210
2018020102
211
2018020103
212
2018020105
213
2018020104
214
2018020106
215
2018020107
216
2018020108
217
2018020109
218
2018020110
219
2018020111
220
2018020112
221
2018020113

2018020594
703
2018020595
704
2018020596
705
2018020597
706
2018020598
707
2018020599
708
2018020600
709
2018020601
710
2018020602
711
2018020603
712
2018020604
713
2018020605
714
2018020606
715
2018020607
716
2018020611
717
2018020608
718
2018020609
719
2018020610
720
2018020612
721
2018020613
722
2018020614
723
2018020615
724
2018020616
725
2018020617
726
2018020618
727
2018020619
728
2018020620
729
2018020621
730
2018020622
731
2018020623
732
2018020625
733
2018020624
734
2018020626
735
2018020627
736
2018020628
737
2018020629
738
2018020630
739
2018020631
740
2018020632
741
2018020633
742
2018020634
743
2018020635
744
2018020636
745
2018020637
746
2018020638
747
2018020639
748
2018020640
749
2018020641
750
2018020642
751
2018020643
752
2018020644
753
2018020647
754
2018020645
755
2018020646
756
2018020648
757
2018020649
758
2018020650
759
2018020651
760
2018020652
761
2018020653
762
2018020654
763
2018020655
764
2018020656
765
2018020657
766
2018020658
767
2018020659
768
2018020660

2018021126
1234
2018021122
1235
2018021127
1236
2018021128
1237
2018021129
1238
2018021130
1239
2018021131
1240
2018021132
1241
2018021133
1242
2018021134
1243
2018021135
1244
2018021136
1245
2018021137
1246
2018021138
1247
2018021139
1248
2018021140
1249
2018021141
1250
2018021142
1251
2018021143
1252
2018021144
1253
2018021145
1254
2018021146
1255
2018021147
1256
2018021148
1257
2018021149
1258
2018021150
1259
2018021151
1260
2018021152
1261
2018021153
1262
2018021154
1263
2018021155
1264
2018021156
1265
2018021157
1266
2018021158
1267
2018021159
1268
2018021160
1269
2018021161
1270
2018021162
1271
2018021163
1272
2018021164
1273
2018021165
1274
2018021166
1275
2018021167
1276
2018021168
1277
2018021169
1278
2018021170
1279
2018021171
1280
2018021172
1281
2018021173
1282
2018021174
1283
2018021175
1284
2018021176
1285
2018021177
1286
2018021178
1287
2018021179
1288
2018021180
1289
2018021181
1290
2018021182
1291
2018021183
1292
2018021184
1293
2018021185
1294
2018021186
1295
20180211

**For the Play Data We Haver X,Y but they're not standardized based on the direction the team is attacking**

We need to grab the "Rink Side" for each team for each game and merge it with our plays data set

In [7]:
def get_rinkside(df):
    rinkside_df = pd.DataFrame(columns = ['Period', 'RinkSide_Home', 'RinkSide_Away', 'GameID', 'Home', 'Away'])
    for game in df['GameID']:
        #Get Events for a game
        r = requests.get('https://statsapi.web.nhl.com/api/v1/game/'+ str(game) + '/feed/live')
        data = r.json()

        period = []
        home = []
        away = []
        for p in data['liveData']['linescore']['periods']:
            try:
                period.append(p['num'])
            except:
                period.append('None')
            try:
                home.append(p['home']['rinkSide'])
            except:
                home.append('None')
            try:
                away.append(p['away']['rinkSide'])
            except:
                away.append('None')

        game_df = pd.DataFrame({'Period': period, 'RinkSide_Home': home, 'RinkSide_Away': away})
        game_df['GameID'] = game
        game_df['Home'] = data['liveData']['linescore']['teams']['home']['team']['name']
        game_df['Away'] = data['liveData']['linescore']['teams']['away']['team']['name']

        #print("Done with " + str(game))
        rinkside_df = pd.concat([rinkside_df, game_df])
        
    home_rinkside = pd.melt(rinkside_df, id_vars=['GameID', 'Period', 'RinkSide_Home'], value_vars=['Home'])
    home_rinkside.columns = ['GameID', 'Period', 'Side', 'Home_Away', 'Event_Team']
    away_rinkside = pd.melt(rinkside_df, id_vars=['GameID', 'Period', 'RinkSide_Away'], value_vars=['Away'])
    away_rinkside.columns = ['GameID', 'Period', 'Side', 'Home_Away', 'Event_Team']
    
    rinkside_df = pd.concat([home_rinkside, away_rinkside])
    
    return rinkside_df

In [8]:
#Get the Rink Sides for Each Game
rinkside_2018 = get_rinkside(game_results_18)
rinkside_2019 = get_rinkside(game_results_19)

In [9]:
'''
Don't really need this function anymore since if it's a shot then it is not a goal or a blocked shot, 
which means it's a save.
'''
# def event2(row):
#     if 'save' in row['Event_Description'].lower():
#         return 'Save'
#     else:
#         return row['Event']

"\nDon't really need this function anymore since if it's a shot then it is not a goal or a blocked shot, \nwhich means it's a save.\n"

In [21]:
#Merge in the Rinkside Data
pbp_2018 = pbp_2018.merge(rinkside_2018[['GameID', 'Event_Team', 'Period', 'Side']], how='left', on=['GameID', 'Event_Team', 'Period'])
pbp_2019 = pbp_2019.merge(rinkside_2019[['GameID', 'Event_Team', 'Period', 'Side']], how='left', on=['GameID', 'Event_Team', 'Period'])

In [22]:
#Write the output for future use
pbp_2018.to_csv('data/pbp_2018.csv', index=False)
pbp_2019.to_csv('data/pbp_2019_121918.csv', index=False)

In [23]:
pbp_2018['Season'] = '2017-2018'
pbp_2019['Season'] = '2018-2019'

In [24]:
combined = pd.concat([pbp_2018, pbp_2019])

In [25]:
len(combined)

706487

In [27]:
shots = combined[combined['Event_Type_ID'] == 'SHOT']

In [28]:
def adjusted_x(row):
    if (row['Event_Team'] == row['Home']) and (row['Period'] in ['1', '3', '5']):
        if row['Side'] == 'Right':
            return row['x']*-1
        else:
            return row['x']
    elif (row['Event_Team'] == row['Home']) and (row['Period'] in ['2', '4']):
        if row['Side'] == 'Left':
            return row['x']*-1
        else:
            return row['x']
    elif (row['Event_Team'] == row['Away']) and (row['Period'] in ['1', '3', '5']):
        if row['Side'] == 'Left':
            return row['x']*-1
        else:
            return row['x']
    elif (row['Event_Team'] == row['Away']) and (row['Period'] in ['2', '4']):
        if row['Side'] == 'Right':
            return row['x']*-1
        else:
            return row['x']
    else:
        return row['x']
    
def adjusted_y(row):
    if (row['Event_Team'] == row['Home']) and (row['Period'] in ['1', '3', '5']):
        if row['Side'] == 'Right':
            return row['y']*-1
        else:
            return row['y']
    elif (row['Event_Team'] == row['Home']) and (row['Period'] in ['2', '4']):
        if row['Side'] == 'Left': 
            return row['y']*-1
        else:
            return row['y']
    elif (row['Event_Team'] == row['Away']) and (row['Period'] in ['1', '3', '5']):
        if row['Side'] == 'Left':
            return row['y']*-1
        else:
            return row['y']
    elif (row['Event_Team'] == row['Away']) and (row['Period'] in ['2', '4']):
        if row['Side'] == 'Right':
            return row['y']*-1
        else:
            return row['y']
    else:
        return row['y']

In [29]:
combined['y_stdz'] = combined.apply(lambda row: adjusted_y(row), axis=1)
combined['x_stdz'] = combined.apply(lambda row: adjusted_x(row), axis=1)

In [30]:
combined['y_normal'] = combined.apply(lambda row: row['y_stdz'] 
                                        if row['Period'] in ['1', '3', '5'] else row['y_stdz']*-1, axis=1)
combined['x_normal'] = combined.apply(lambda row: row['x_stdz'] 
                                        if row['Period'] in ['1', '3', '5'] else row['x_stdz']*-1, axis=1)

In [31]:
combined['Event_Team_Adj'] = combined.apply(lambda row: row['Event_Team'] if row['Event'] != 'Giveaway'
                                             else (row['Home'] if row['Event_Team'] == row['Away'] else row['Away']), axis=1)

In [32]:
combined['y_same'] = combined.apply(lambda row: row['y'] 
                                        if row['Side'] == 'right' else row['y']*-1, axis=1)
combined['x_same'] = combined.apply(lambda row: row['x'] 
                                        if row['Side'] == 'right' else row['x']*-1, axis=1)

In [33]:
#Find the 'Zones'

#Forward X >=-86, X <= -65
#Middle X > -65, X <= -51
#Back X > -51, X<=-28

#Center Y >= -3, Y <= 3
#Right Y> 3, Y<=18
#Right Board Y > 18
#Left Y<-3, Y >=-18
#Left Board Y < -18

def get_shot_zone_x(x):
    if (x >= -86) & (x <= -65):
        return "Forward"
    elif (x> -65) & (x <= -51):
        return "Middle"
    elif (x > -51) & (x <= -28):
        return "Back"
    else:
        return "Other"
        
def get_shot_zone_y(y):
    if (y >= -3) & (y <= 3):
        return "Center"
    elif (y > 3) & (y <= 18):
        return "Right"
    elif (y > 18):
        return "Right Board"
    elif (y < -3) & (y >= -18):
        return "Left"
    elif (y < -18):
        return "Left Board"
    
def get_shot_zone(row):
    if (row['zone_x'] == 'Forward') & (row['zone_y'] == 'Center'):
        return "Center Net Front"
    elif (row['zone_x'] == 'Forward') & (row['zone_y'] == 'Right'):
        return "Right Net Front"
    elif (row['zone_x'] == 'Forward') & (row['zone_y'] == 'Left'):
        return "Left Net Front"
    elif (row['zone_x'] == 'Middle') & (row['zone_y'] == 'Center'):
        return "Center Top Circle"
    elif (row['zone_x'] == 'Middle') & (row['zone_y'] == 'Left'):
        return "Left Top Circle"
    elif (row['zone_x'] == 'Middle') & (row['zone_y'] == 'Right'):
        return "Right Top Circle"
    elif (row['zone_x'] == 'Back') & (row['zone_y'] == 'Left Board'):
        return "Left Point"
    elif (row['zone_x'] == 'Back') & (row['zone_y'] == 'Right Board'):
        return "Right Point"
    elif (row['zone_x'] == 'Back') & ((row['zone_y'] == 'Center') | (row['zone_y'] == 'Left') | (row['zone_y'] == 'Right')):
        return "Center Blue Line"
    elif ((row['zone_x'] == 'Middle') | (row['zone_x'] == 'Forward')) & (row['zone_y'] == 'Left Board'):
        return "Left Board"
    elif ((row['zone_x'] == 'Middle') | (row['zone_x'] == 'Forward')) & (row['zone_y'] == 'Right Board'):
        return "Right Board"
    else:
        return "Other"
    
    
    
    

In [34]:
combined['zone_x'] = combined['x_same'].apply(lambda x: get_shot_zone_x(x))
combined['zone_y'] = combined['y_same'].apply(lambda x: get_shot_zone_y(x))

In [35]:
combined['shot_zone'] = combined.apply(lambda row: get_shot_zone(row), axis=1)

In [36]:
#Add an Opposition Team
combined['Opposition Team'] = combined.apply(lambda row:
                                              row['Away'] if row['Event_Team_Adj'] == row['Home']
                                              else row['Home'], axis=1)

In [37]:
#Let's Add Shot Types from http://blog.war-on-ice.com

#Rebounds = shots taken within 3 seconds of another shot
#Rush Shots = shots taken within 4 seconds of an event in another zone

combined['Shifted_Event_Type_ID'] = combined.groupby('GameID')['Event_Type_ID'].shift(1)
combined['Shifted_shot_zone'] = combined.groupby('GameID')['shot_zone'].shift(1)
combined['Shifted_Time_Remaining'] = combined.groupby('GameID')['Time_Remaining'].shift(1)

In [38]:
combined['Time_Remaining'] = pd.to_datetime(combined['Time_Remaining'], format='%M:%S')
combined['Shifted_Time_Remaining'] = pd.to_datetime(combined['Shifted_Time_Remaining'], format='%M:%S')

In [39]:
combined['Event_Time_Difference'] = combined['Shifted_Time_Remaining'] - combined['Time_Remaining']
combined['Event_Time_Difference'] = combined['Event_Time_Difference'].dt.total_seconds()

In [40]:
included_events = ['FACEOFF', 'HIT', 'MISSED_SHOT', 'GIVEAWAY',
                  'TAKEAWAY']
def shot_types(row):
    #Find Rush Shots First - a shot taken within 4 seconds of an event in a different zone
    if (row['Event_Type_ID'] == 'SHOT') and (row['shot_zone'] != row['Shifted_shot_zone']) and (row['Shifted_Event_Type_ID'] in included_events) and (row['Event_Time_Difference'] <= 4.0):
        return 'Rush Shot'
    #Rebounds
    elif (row['Event_Type_ID'] == 'SHOT') and (row['Shifted_Event_Type_ID'] in ['SHOT', 'BLOCKED_SHOT']) and (row['Event_Time_Difference'] <= 3.0):
        return 'Rebound Shot'
    elif (row['Event_Type_ID'] == 'SHOT'):
        return 'Shot'
    else:
        return 'Non-Shot Event'

In [41]:
combined['Shot_Context'] = combined.apply(lambda row: shot_types(row), axis=1) 

In [42]:
#Write Data To File
combined.to_csv("data/combined_seasons.csv")